## Step 1: Load Libraries

In [1]:
# Install all required packages
!pip install pandas krippendorff statsmodels scikit-learn seaborn matplotlib numpy nltk


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import pandas as pd
import krippendorff
from statsmodels.stats.inter_rater import fleiss_kappa
from sklearn.metrics import cohen_kappa_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa
import itertools
import seaborn as sns
from scipy.stats import entropy
from sklearn.cluster import KMeans
from scipy.special import rel_entr
from scipy.stats import entropy
import itertools
from collections import Counter
import pandas as pd
import ast 
from statsmodels.stats.inter_rater import fleiss_kappa
from itertools import product
from nltk.metrics import agreement
from nltk.metrics import masi_distance
from collections import defaultdict
import json

In [3]:
df = pd.read_csv("./gold_dataset.csv")
df.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.text-eng,Input.label,Input.parameter,Input.parameter_num_labels,Answer.manifesto_class.labels,Approve,Reject
0,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
1,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (238/239),100% (19/19),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
2,39N6W9XWS05H65P9I0HAUP0E2RZGYY,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Madam President, Mr Posselt, I do not really t...",0,Par303_Governmental and Administrative Efficiency,2,[],NaN,NaN
3,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (152/152),100% (46/46),100% (46/46),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Keynesian Demand Management"",""Market regulat...",NaN,NaN
4,3P7RGTLO71VLC9NPSLO9PVA06HDAKL,3BRS3IGD63EV308CIXT46SFWM41TR3,[ACSS] Text labeling Manifesto,Identify the most relevant labels to describe ...,"labeling, text, manifesto",$1.00,Mon Apr 14 05:38:01 PDT 2025,3,BatchId:414859;OriginalHitTemplateId:921587261;,3600,...,100% (287/287),100% (75/75),100% (19/19),"Mr President, ladies and gentlemen, first of a...",1,Par405_Corporatism,2,"[""Government and administrative efficiency"",""M...",NaN,NaN


#### **Data Cleaning and Pre-Proccessing**

In [4]:
# only keep needed columns and rename 
df_clean = df[['HITId', 'WorkerId', 'Answer.manifesto_class.labels']].copy() #remove unnecessary columns
df_clean.rename(columns={
    'HITId': 'speech_id',
    'WorkerId': 'worker_id',
    'Answer.manifesto_class.labels': 'answer'
}, inplace=True)
df_clean = df_clean.sort_values(by=['speech_id', 'worker_id']).reset_index(drop=True)

# Final preview
df_clean

,speech_id,worker_id,answer
0,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1NBQ61Y6KO3O9,"[""Market regulation"",""Middle Class & Professio..."
1,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A1X47COW2Y9SEL,"[""Market regulation""]"
2,301KG0KXAZ355GRD9LSX6LBA2CHH2H,A335R4YE2E34H6,"[""Market regulation""]"
3,302OLP89EMP3TBB1087B1CXS184CAA,A1NBQ61Y6KO3O9,"[""Government and administrative efficiency"",""E..."
4,302OLP89EMP3TBB1087B1CXS184CAA,A1X47COW2Y9SEL,"[""Economic Goals""]"
...,...,...,...
655,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A1X47COW2Y9SEL,"[""Market regulation"",""Government and administr..."
656,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,A3GS2NTQ4XU059,"[""Free Market Economy""]"
657,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1BQ37ZGUM16XI,"[""Market regulation""]"
658,3ZXV7Q5FKY6YP2S9BOT3FD8JYOHCFN,A1X47COW2Y9SEL,"[""Market regulation"",""Anti-Growth Economy""]"


define workers and 'categories' - the list of categories they chose from for answers

In [ ]:
workers = [
    'A1BQ37ZGUM16XI', 
    'A1NBQ61Y6KO3O9', 
    'A1X47COW2Y9SEL', 
    'A335R4YE2E34H6', 
    'A3GS2NTQ4XU059', 
    'APXGSUB250NVH'
]

with open("unique_categories.json", "r") as f:
    unique_categories = json.load(f)
unique_categories
 

['Market regulation',
 'Middle Class & Professional Groups',
 'Government and administrative efficiency',
 'Economic Planning',
 'Economic Goals',
 'Anti-Growth Economy',
 'Economic Growth',
 'Free Market Economy',
 'Incentives',
 'Keynesian Demand Management',
 'Corporatism / Mixed Economy',
 'Technology and Infrastructure',
 'Labor Groups Positive',
 'Labor Groups Negative']

Now, we try to pivot the data to create different dataframes for every speech !!

In [17]:

df_clean['answer'] = df_clean['answer'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_filtered = df_clean[df_clean['answer'].apply(lambda x: isinstance(x, list) and len(x) > 0)] 

speech_dfs = {} # result dict

for speech_id, group in df_filtered.groupby('speech_id'):
    if len(group) != 3:
        continue  # skip if not exactly 3 annotations

    df = pd.DataFrame(index=unique_categories)  # rows = categories
    for i, (_, row) in enumerate(group.iterrows()):
        worker_label = f'worker{i+1}'
        df[worker_label] = df.index.map(lambda category: 1 if category in row['answer'] else 0)

    speech_dfs[speech_id] = df

        
speech_dfs['306996CF7J2OPI6VQPINFRY7FTYB1E']

,worker1,worker2,worker3
Market regulation,1,0,0
Middle Class & Professional Groups,0,0,0
Government and administrative efficiency,0,1,0
Economic Planning,0,0,1
Economic Goals,0,0,0
Anti-Growth Economy,1,0,0
Economic Growth,0,0,1
Free Market Economy,0,0,0
Incentives,0,0,0
Keynesian Demand Management,0,0,0


## Run Fleiss Kappa

Run on every speech and create a new df to see the results

In [18]:


# Store results
kappa_scores = []

for speech_id, df in speech_dfs.items():
    # Each row is a category → count number of 1s and 0s across workers
    count_matrix = []
    for _, row in df.iterrows():
        counts = [list(row).count(0), list(row).count(1)]  # [num 0s, num 1s]
        count_matrix.append(counts)

    count_array = np.array(count_matrix)

    # Compute Fleiss' Kappa
    try:
        kappa = fleiss_kappa(count_array)
    except Exception as e:
        print(f"Error on {speech_id}: {e}")
        kappa = np.nan

    kappa_scores.append({
        'speech_id': speech_id,
        'fleiss_kappa': kappa
    })

# Convert to DataFrame
df_kappa = pd.DataFrame(kappa_scores)


In [ ]:
df_kappa

,speech_id,fleiss_kappa
0,301KG0KXAZ355GRD9LSX6LBA2CHH2H,0.723684
1,306996CF7J2OPI6VQPINFRY7FTYB1E,-0.135135
2,3087LXLJ79YEVAXTZ80JKC6IPZKF0X,0.485714
3,308KJXFUKEOKFSDCMU21QMXWPU4ATR,0.416667
4,30EMX9PEW71PC59J9LA0AV46S08KS5,-0.105263
...,...,...
141,3YZ7A3YHSSBMUMOG2Y6U86LOVAD5S5,0.151515
142,3ZUE82NE1XJWDZ2WUCDYDRC08AKF8Y,0.222222
143,3ZURAPD29V5E2ZIBVR4JYHPSURQF10,0.485714
144,3ZXNP4Z3AE3EDD796IOQU4LPUM0L7M,0.142857
